In [1]:
import pandas as pd

In [2]:
# Constantes
PREVIOUS_RESULTS_USED = 5

In [3]:
raw_dfs = {
    'races': pd.read_parquet('.data_parquet/races.parquet'),
    'results': pd.read_parquet('.data_parquet/results.parquet'),
    'driver_standings': pd.read_parquet('.data_parquet/driver_standings.parquet'),
    'constructor_standings': pd.read_parquet('.data_parquet/constructor_standings.parquet'),
    'qualifying': pd.read_parquet('.data_parquet/qualifying.parquet')
}
display(raw_dfs['races'])

,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time,datetime
raceId,,,,,,,,,,,,,,,,,,
1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,2009-03-29 06:00:00
2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,2009-04-05 09:00:00
3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,2009-04-19 07:00:00
4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,2009-04-26 12:00:00
5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,2009-05-10 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,2023,18,69,United States Grand Prix,2023-10-22,19:00:00,https://en.wikipedia.org/wiki/2023_United_Stat...,2023-10-20,17:30:00,2023-10-21,18:00:00,\N,\N,2023-10-20,21:00:00,2023-10-21,22:00:00,2023-10-22 19:00:00
1117,2023,19,32,Mexico City Grand Prix,2023-10-29,20:00:00,https://en.wikipedia.org/wiki/2023_Mexico_City...,2023-10-27,18:30:00,2023-10-27,22:00:00,2023-10-28,17:30:00,2023-10-28,21:00:00,\N,\N,2023-10-29 20:00:00
1118,2023,20,18,São Paulo Grand Prix,2023-11-05,17:00:00,https://en.wikipedia.org/wiki/2023_S%C3%A3o_Pa...,2023-11-03,14:30:00,2023-11-04,14:30:00,\N,\N,2023-11-03,18:00:00,2023-11-04,18:30:00,2023-11-05 17:00:00


In [4]:
races_by_number = raw_dfs['races'][['year', 'round']].sort_values(['year', 'round'])
races_by_number['number'] = range(1, len(races_by_number) + 1)
races_number = races_by_number[['number']]
races_df_with_number = raw_dfs['races'].join(races_by_number['number'], on='raceId')
display(races_df_with_number)

,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time,datetime,number
raceId,,,,,,,,,,,,,,,,,,,
1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,2009-03-29 06:00:00,804
2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,2009-04-05 09:00:00,805
3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,2009-04-19 07:00:00,806
4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,2009-04-26 12:00:00,807
5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,2009-05-10 12:00:00,808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,2023,18,69,United States Grand Prix,2023-10-22,19:00:00,https://en.wikipedia.org/wiki/2023_United_Stat...,2023-10-20,17:30:00,2023-10-21,18:00:00,\N,\N,2023-10-20,21:00:00,2023-10-21,22:00:00,2023-10-22 19:00:00,1097
1117,2023,19,32,Mexico City Grand Prix,2023-10-29,20:00:00,https://en.wikipedia.org/wiki/2023_Mexico_City...,2023-10-27,18:30:00,2023-10-27,22:00:00,2023-10-28,17:30:00,2023-10-28,21:00:00,\N,\N,2023-10-29 20:00:00,1098
1118,2023,20,18,São Paulo Grand Prix,2023-11-05,17:00:00,https://en.wikipedia.org/wiki/2023_S%C3%A3o_Pa...,2023-11-03,14:30:00,2023-11-04,14:30:00,\N,\N,2023-11-03,18:00:00,2023-11-04,18:30:00,2023-11-05 17:00:00,1099


In [5]:
# Parâmetros pensados
# - Últimas posições de largada do piloto
# - Últimas posições de chegada do piloto
# - Histórico de classificação do piloto
# - Posição "média" de largada da equipe nas últimas corridas
# - Posição "média" de chegada da equipe nas últimas corridas
# - Histórico de classificação da equipe
# Agrupando todas as informações necessárias no df de resultados
results_df = raw_dfs['results']
joined_driver_standings_df = (
    raw_dfs['driver_standings'][['position']].rename(
        columns={'position': 'driver_standings_position'}
    )
)
joined_constructor_standings_df = (
    raw_dfs['constructor_standings'][['position']].rename(
        columns={'position': 'constructor_standings_position'}
    )
)
joined_race_df = (
    races_df_with_number[['number', 'year']]
)
results_df = results_df.join(joined_driver_standings_df)
results_df = results_df.join(joined_constructor_standings_df, on=['raceId', 'constructorId'])
results_df = results_df.join(joined_race_df)
results_df = results_df.reset_index()
results_df = results_df.set_index(['driverId', 'number'])
results_df['driverId'] = results_df.index.get_level_values(0)
results_df = results_df.sort_index()
results_df['driverId_shifted'] = results_df.groupby(level=0)['driverId'].shift(PREVIOUS_RESULTS_USED)
results_df['driverId_equals'] = results_df['driverId'] == results_df['driverId_shifted']
results_df['year_shifted'] = results_df['year'].shift(PREVIOUS_RESULTS_USED)
results_df['year_equals'] = results_df['year'] == results_df['year_shifted']
display(results_df)

raceId  constructorId  grid  position  points  laps  \
driverId number                                                        
1        769         36              1     4       3.0     6.0    58   
         770         37              1     4       2.0     8.0    56   
         771         38              1     2       2.0     8.0    57   
         772         39              1     4       2.0     8.0    65   
         773         40              1     2       2.0     8.0    78   
...                 ...            ...   ...       ...     ...   ...   
858      1087      1106              3    18       NaN     0.0     6   
         1088      1107              3    18      13.0     0.0    70   
         1089      1108              3    14      11.0     0.0    52   
         1090      1109              3    20       NaN     0.0    67   
         1091      1110              3    18      17.0     0.0    44   

                 milliseconds  fastestLap  rank fastestLapTime  ... statusId  \
driverId number                                                 ...            
1        769        5147365.0        20.0   3.0       1:26.351  ...        1   
         770        5552487.0        22.0   1.0       1:36.701  ...        1   
         771        5609875.0        18.0   2.0       1:34.270  ...        1   
         772        5503020.0        20.0   2.0       1:22.876  ...        1   
         773        6033424.0        28.0   2.0       1:15.372  ...        1   
...                       ...         ...   ...            ...  ...      ...   
858      1087             NaN         4.0  20.0       1:18.437  ...        5   
         1088             NaN        45.0  12.0       1:09.611  ...       11   
         1089       5140570.0        42.0  13.0       1:31.699  ...        1   
         1090             NaN        36.0  12.0       1:23.496  ...       31   
         1091       5054926.0        37.0   9.0       1:50.486  ...        1   

                 fastestLapMillis  driver_standings_position  \
driverId number                                                
1        769              86351.0                        3.0   
         770              96701.0                        3.0   
         771              94270.0                        3.0   
         772              82876.0                        1.0   
         773              75372.0                        2.0   
...                           ...                        ...   
858      1087             78437.0                       20.0   
         1088             69611.0                       19.0   
         1089             91699.0                       19.0   
         1090             83496.0                       19.0   
         1091            110486.0                       19.0   

                 constructor_standings_position  year  driverId  \
driverId number                                                   
1        769                               11.0  2007         1   
         770                               11.0  2007         1   
         771                               11.0  2007         1   
         772                               11.0  2007         1   
         773                               11.0  2007         1   
...                                         ...   ...       ...   
858      1087                               9.0  2023       858   
         1088                               9.0  2023       858   
         1089                               7.0  2023       858   
         1090                               7.0  2023       858   
         1091                               7.0  2023       858   

                 driverId_shifted  driverId_equals  year_shifted  year_equals  
driverId number                                                                
1        769                  NaN            False           NaN        False  
         770                  NaN            False           NaN        False  
         771  

In [6]:
results_df[results_df['driverId_equals'] & results_df['year_equals']]

raceId  constructorId  grid  position  points  laps  \
driverId number                                                        
1        774         41              1     1       1.0    10.0    70   
         775         42              1     1       1.0    10.0    73   
         776         43              1     2       3.0     6.0    70   
         777         44              1     1       3.0     6.0    59   
         778         45              1    10       9.0     0.0    59   
...                 ...            ...   ...       ...     ...   ...   
858      1087      1106              3    18       NaN     0.0     6   
         1088      1107              3    18      13.0     0.0    70   
         1089      1108              3    14      11.0     0.0    52   
         1090      1109              3    20       NaN     0.0    67   
         1091      1110              3    18      17.0     0.0    44   

                 milliseconds  fastestLap  rank fastestLapTime  ... statusId  \
driverId number                                                 ...            
1        774        6251292.0        37.0   2.0       1:16.494  ...        1   
         775        5469965.0        20.0   2.0       1:13.222  ...        1   
         776        5486353.0        39.0   4.0       1:16.587  ...        1   
         777        4942447.0        11.0   4.0       1:21.675  ...        1   
         778              NaN        29.0   3.0       1:33.401  ...       11   
...                       ...         ...   ...            ...  ...      ...   
858      1087             NaN         4.0  20.0       1:18.437  ...        5   
         1088             NaN        45.0  12.0       1:09.611  ...       11   
         1089       5140570.0        42.0  13.0       1:31.699  ...        1   
         1090             NaN        36.0  12.0       1:23.496  ...       31   
         1091       5054926.0        37.0   9.0       1:50.486  ...        1   

                 fastestLapMillis  driver_standings_position  \
driverId number                                                
1        774              76494.0                        1.0   
         775              73222.0                        1.0   
         776              76587.0                        1.0   
         777              81675.0                        1.0   
         778              93401.0                        1.0   
...                           ...                        ...   
858      1087             78437.0                       20.0   
         1088             69611.0                       19.0   
         1089             91699.0                       19.0   
         1090             83496.0                       19.0   
         1091            110486.0                       19.0   

                 constructor_standings_position  year  driverId  \
driverId number                                                   
1        774                               11.0  2007         1   
         775                               11.0  2007         1   
         776                               11.0  2007         1   
         777                               11.0  2007         1   
         778                               11.0  2007         1   
...                                         ...   ...       ...   
858      1087                               9.0  2023       858   
         1088                               9.0  2023       858   
         1089                               7.0  2023       858   
         1090                               7.0  2023       858   
         1091                               7.0  2023       858   

                 driverId_shifted  driverId_equals  year_shifted  year_equals  
driverId number                                                                
1        774                  1.0             True        2007.0         True  
         775                  1.0             True        2007.0         True  
         776  

In [7]:
tmp_df = raw_dfs['results']
tmp_df = tmp_df.reset_index()
tmp_df = tmp_df.set_index(['raceId', 'constructorId']).sort_index()
tmp_df = (
    tmp_df
    .groupby(['raceId', 'constructorId'])
    [['grid', 'position']]
    .mean()
)
tmp_df = tmp_df.rename(columns={
    'grid': 'equivalent_grid_constructor',
    'position': 'equivalent_position_constructor'
})
tmp_df

equivalent_grid_constructor  \
raceId constructorId                                
1      1                                     15.0   
       2                                      6.5   
       3                                      8.0   
       4                                     12.0   
       5                                     15.0   
...                                           ...   
1110   117                                    9.5   
       131                                    5.5   
       210                                    8.0   
       213                                   15.0   
       214                                   13.0   

                      equivalent_position_constructor  
raceId constructorId                                   
1      1                                          NaN  
       2                                         12.0  
       3                                          6.0  
       4                                          5.0  
       5                                          7.5  
...                                               ...  
1110   117                                        7.0  
       131                                        5.0  
       210                                       16.5  
       213                                       13.0  
       214                                        9.5  

[12688 rows x 2 columns]

In [8]:
results_df = results_df.join(tmp_df, on=['raceId', 'constructorId'])
results_df

raceId  constructorId  grid  position  points  laps  \
driverId number                                                        
1        769         36              1     4       3.0     6.0    58   
         770         37              1     4       2.0     8.0    56   
         771         38              1     2       2.0     8.0    57   
         772         39              1     4       2.0     8.0    65   
         773         40              1     2       2.0     8.0    78   
...                 ...            ...   ...       ...     ...   ...   
858      1087      1106              3    18       NaN     0.0     6   
         1088      1107              3    18      13.0     0.0    70   
         1089      1108              3    14      11.0     0.0    52   
         1090      1109              3    20       NaN     0.0    67   
         1091      1110              3    18      17.0     0.0    44   

                 milliseconds  fastestLap  rank fastestLapTime  ...  \
driverId number                                                 ...   
1        769        5147365.0        20.0   3.0       1:26.351  ...   
         770        5552487.0        22.0   1.0       1:36.701  ...   
         771        5609875.0        18.0   2.0       1:34.270  ...   
         772        5503020.0        20.0   2.0       1:22.876  ...   
         773        6033424.0        28.0   2.0       1:15.372  ...   
...                       ...         ...   ...            ...  ...   
858      1087             NaN         4.0  20.0       1:18.437  ...   
         1088             NaN        45.0  12.0       1:09.611  ...   
         1089       5140570.0        42.0  13.0       1:31.699  ...   
         1090             NaN        36.0  12.0       1:23.496  ...   
         1091       5054926.0        37.0   9.0       1:50.486  ...   

                driver_standings_position  constructor_standings_position  \
driverId number                                                             
1        769                          3.0                            11.0   
         770                          3.0                            11.0   
         771                          3.0                            11.0   
         772                          1.0                            11.0   
         773                          2.0                            11.0   
...                                   ...                             ...   
858      1087                        20.0                             9.0   
         1088                        19.0                             9.0   
         1089                        19.0                             7.0   
         1090                        19.0                             7.0   
         1091                        19.0                             7.0   

                 year  driverId  driverId_shifted  driverId_equals  \
driverId number                                                      
1        769     2007         1               NaN            False   
         770     2007         1               NaN            False   
         771     2007         1               NaN            False   
         772     2007         1               NaN            False   
         773     2007         1               NaN            False   
...               ...       ...               ...              ...   
858      1087    2023       858             858.0             True   
         1088    2023       858             858.0             True   
         1089    2023       858             858.0             True   
         1090    2023       858             858.0             True   
         1091    2023       858             858.0             True   

                 year_shifted  year_equals  equivalent_grid_constructor  \
driverId number                                                           
1        769              NaN        False                          3.0   
         770      

In [9]:
tmp2_df = (
    results_df[[
        'equivalent_grid_constructor',
        'equivalent_position_constructor',
        'position',
        'grid'
    ]]
    .rolling(PREVIOUS_RESULTS_USED)
    .agg(['mean', 'std'])
    .rename(columns={
        'position': 'position_driver',
    'grid': 'grid_driver'
    }
    )
    .head(10)
)
tmp2_df.columns = ['_'.join(col).strip() for col in tmp2_df.columns.values]
tmp2_df

equivalent_grid_constructor_mean  \
driverId number                                     
1        769                                  NaN   
         770                                  NaN   
         771                                  NaN   
         772                                  NaN   
         773                                  2.7   
         774                                  2.4   
         775                                  2.1   
         776                                  2.7   
         777                                  2.5   
         778                                  3.4   

                 equivalent_grid_constructor_std  \
driverId number                                    
1        769                                 NaN   
         770                                 NaN   
         771                                 NaN   
         772                                 NaN   
         773                            0.670820   
         774                            0.821584   
         775                            0.821584   
         776                            1.955761   
         777                            1.968502   
         778                            2.382226   

                 equivalent_position_constructor_mean  \
driverId number                                         
1        769                                      NaN   
         770                                      NaN   
         771                                      NaN   
         772                                      NaN   
         773                                      2.3   
         774                                      2.6   
         775                                      2.6   
         776                                      2.9   
         777                                      2.9   
         778                                      3.6   

                 equivalent_position_constructor_std  position_driver_mean  \
driverId number                                                              
1        769                                     NaN                   NaN   
         770                                     NaN                   NaN   
         771                                     NaN                   NaN   
         772                                     NaN                   NaN   
         773                                0.836660                   2.2   
         774                                1.140175                   1.8   
         775                                1.140175                   1.6   
         776                                1.557241                   1.8   
         777                                1.557241                   2.0   
         778                                1.557241                   3.4   

                 position_driver_std  grid_driver_mean  grid_driver_std  
driverId number                                                          
1        769                     NaN               NaN              NaN  
         770                     NaN               NaN              NaN  
         771                     NaN               NaN              NaN  
         772                     NaN               NaN              NaN  
         773                0.447214               3.2         1.095445  
         774                0.447214               2.6         1.341641  
         775                0.547723               2.0         1.224745  
         776                0.836660               2.0         1.224745  
         777                1.000000               1.4         0.547723  
         778                3.286335               3.0         3.937004